In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_transformers import get_kobert_model


import nltk
from itertools import product
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from sklearn.preprocessing import normalize
import pandas as pd


from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')

# 초기화
from konlpy.tag import Okt


import nltk
import jpype
import jpype.imports
from jpype.types import *

from transformers import BertModel, BertTokenizer
from transformers import AutoModel
from sentence_transformers import SentenceTransformer


from bertopic import BERTopic

from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from ugtm import eGTM
import umap
import hdbscan

c:\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# NLTK 설정 및 리소스 다운로드


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
patent = pd.read_excel('patentdata_v.0.4.xls')

In [4]:
summ = patent['청구항']

In [601]:
# 불용어 목록 (수정 - 보고서 기반 불용어 제거)

stopword = {
    
    
    '및', '또는', '시스템', '프로세스', '모델', '삭제', '청구항', '추출', '발명', '문서', '구조', '화면', '반도체', '기능', '프로그램', '연결', '시작', '운전', '권한', '구동', '조회', '현황', '내용', '점자', '보건',
    '계측', '제조', '포함', '출력', '송신', '저장', '생산', '전달', '방식', '자료', '환경', '효과', '조절', '들다', '동일하다', '발음', '음주','투과','버스','플랜트','블록','소자','출입문','수소','서브', 
    '실행', '수집', '특성', '발생', '구성', '작동', '장비', '접속', '고정', '운영', '전환', '획득', '디바이스', '할당', '스펙트럼', '식별', '표면온도', '차단', '전방', '인증', '진입', 
    '구현', '수행', '제한', '효율', '형성', '절차', '변환', '배치', '발견', '감소', '유도', '저감', '계산', '전개', '상황', '통화', '보행', '집중', '회전', '현재상황', '좌우','위험하다','속도','축적','분해','장소','아릴','치환','여기','임의','테로','원자','화합물','모션','미러','말단',
    '방출', '형태', '적용', '최적화', '하다', '상기', '되다', '작업', '있다', '관리', '으로', '보다', '화학', '객체', '아크', '볼륨', '진동', '소화기', '애', '플러그','어셈블리','채널','캐쉬','투입',
    '대한', '모듈', '조물', '되어다', '현장', '은', '는', '를', '이용', '에', '관하여', '관해', '것', '따르다', '체험', '미세먼지','주변', '사면','차원','사업자',
    '의', '예', '각종', '산업', '몸', '사용자', '채', '요소', '자', '에게', '정량', '정성', '적', '전압', '치유', '오브젝트', '무선통신', '사용', '광고',
    '가다', '제공', '로부터', '인접', '구', '영향', '미치다', '요인', '선별', '계측값', '인접물', '가시설', '것으로서', '농도', '근접', '감성', '변위', '석기', '자선',
    '계획', '서다', '감리자', '확인', '이루어지다', '부', '자격', '인적', '부가', '물', '물의', '이', '면', '별로', '스크램블링', '박스','귀마개', 
    '각각', '관련', '위', '공사', '사업', '등', '용이하다', '이로', '수', '인하다', '노동', '강도', '줄이다', '규정', '가드', '포스트', '래커', '가스',
    '준수', '오다', '쓰러지다', '공간', '설치', '실시', '실시예', '점유', '율', '협업', '이다', '전용', '피뢰침', '코드', '승인', '센터', 
    '늘다', '시설', '측정', '수치', '기준', '중요하다', '남아', '흙막이', '설', '와', '통해', '보고서', '이고', '가드 포스트', '질의', '정정',
    '과', '가지다', '자의', '또한', '에서는', '일', '의하다', '생', '성', '대해', '복수', '으로써', '계', '가시', '온도', '메인보드', 
    '관', '부와', '측', '값', '얻다', '미지', '물론', '주다', '부르다', '이르다', '바다', '도록', '전후', '동', '이동식','전복', '치료','외피', '두부', '파장', '과정', '크루', '태블릿', '케이', '어플', '튜브',
    '축', '관통', '공', '유', '입구', '구르다', '구로', '사이', '대역', '설정', '현상', '검', '제', '구의', '움직임 감지', '감지 센서',
    '내', '해', '추다', '적어도', '해당', '산출', '물질', '가설', '후', '벗어나다', '제시', '단말', '자다', '범위', '영역', '안전성', '시공', '안전성 평가', '이동형 시공',
    '경우', '표시', '통지', '특징', '위해', '철거', '원', '방향', '이동성', '누적', '지시', '레이', '계통', '보조', '감성','위험',
    '모으다', '불다', '이중화', '마', '그네', '틱', '콘텍트', '어느', '한쪽', '일부', '지점', '다른', '단락', '상거동', '질문', '대상', '수용', '하천', '수배', '전반', '석면',
    '폐', '간', '그룹', '건전하다', '동시에', '원천적', '활용', '기반', '혼합', '정이', '때', '각',  '미분', '미션', '안심', '필터', '비실', '구획', '덮개'
    '상황', '시', '가능하다', '장인', '신속하다', '폴', '다양하다', '연결하다', '개선', '조건', '알다', '줄', '파라미터', '통합', '터널',
    '레스트', '개', '상세', '빈번', '자연재해', '저', '감', '과제', '붕괴', '강하다', '피해', '재산', '로고', '용품', '길다', '공종','승강기',
    '유역', '범', '검토', '적극', '화', '마다', '구비', '을', '으로서', '한', '함', '역', '용', '기', '에서', '하나', '어부', '하', '거나', '기', '속', '서', '가', '한', '에서', '모', '로', '격', '각각으로', '부터',
    '함', '반', '대하', '여', '어', '로', '하고', '흐르다', '가', '류', '중', '화하다', '지역', '뿐', '만', '시키다', '라돈', '안내', '거리', '시나리오', '확산', '소화', '대다', '사람', 
    '화상', '패드', '미리', '디스플레이', '클러스터', '물품', '서명', '노드', '스트림', '소방', '출입', '으로부터', '방송', '비교', '개방', '전류','전력','스케줄링',
    '아니다', '기존', '안전하다', '지', '중', '하이', '식', '노면', '그리고', '그', '으로는', '명세서', '이러하다', '통', '받다', '부로', '신', '필요하다', '미세먼지 농도',
    '사', '고원', '재해', '원인', '사고', '분류', '별', '고르다', '바', '오', '탐미', '타다', '영', '상인', '인', '재', '셋', '하영', '영',
    '형', '끈', '주', '이나', '크다', '웨어', '누', '올바르다', '갖다', '조', '치가', '쉬다', '유닛', '다', '더욱', '종', '들', '다수', '특히', '밉다', '작', '업도', '업자', '들', '않다', '머신',
    '고', '청', '단', '말로', '성하다', '층', '항상', '러블', '업', '부터의', '가망', '부의', '피', '가의', '양', '나', '물이', '부위', '정해석',
    '본체', '고리', '단계', '체결', '여부', '프로젝트', '부분', '고장', '문제점', '체내', '건설 프로젝트', '동구', '보호', '보도', '전송', '수신', '디지털', '손상', '발전', '점검', '판단', '도면', '파악', '개구', '선택', '이동식 크레인', '근로자', '패턴', '하천 제방', '금속 배관',
    '에스', '처지', '지피다', '지피다 에스', '크레인', '서버', '증강현실 영상', '인자', '운항 선박', '선박 운항', '선박 항해', '무선 중계기', '시점 이전', '이전 이후', '라이트 인터페이스', '이전', '근거', '조치', '인증 인증', '촬영', '시점 이전', '이전 이후', '인식 인증', '클라이언트',
    '스마트', '외부', '침입', '퇴치', '배', '지부', '대체', '시공사', '발주', '처', '연', '징후', '대처', '은닉', '특정', '메모리', '와의', '요청', '대응', '응답', '송출', '리스트', '소정', '구간', '내의', '연', '신하', '구체', '구역', '성함', '양상',
    '규제', '유해', '역사', '어댑터', '동작', '결과', '사각', '지수', '내지', '모드', '시급', '탐지', '실내', '게이트웨이', '공정', '개시', '멜빵', '부착', '체크', '전원', '공급', '유해', '색', '변하다', '염료', '색상', '유출', '개시', '정원', '기질', '쾌적', '화재',
    '공장', '빌딩', '아파트', '같다', '대형', '건물', '차량', '위생', '절감', '쾌적하다', '같다', '특', '고압', '특 고압', '입력', '자동', '백업', '항목', '지하철', '명령', '피난', '화염', '재다', '연기', '신호', '통신',
    '컴퓨터', '매체', '학습', '성부', '심층', '관측', '제품', '업체', '번호', '일자', '유효', '기간', '조사', '조국', '업체', '구분', '명', '게',  '인지', '인식', '기초', '부재', '등급', '진단', '장착', '기계', '하량', '시도', '신부', '뛰어나다',
    '청구', '당사자', '삭제', '상호작용', '서상기', '휴대', '관리자', '항', '알킬', '프레임', '내부', '지진', '서비스', '변화', '현재', '시간', '이벤트' ,'결합', '장애', '주파수', '부감', '치르다', '표준', '구축',
    '주행', '자율', '동체', '나노', '전자', '부스', '고유', '액세스', '해제', '코딩', '원단', '전장', '잠금장치', '융합', '출사', '출하', '기기', '부여','임대', '입출력','수단','종단','임차','타입', '산소','커튼','피뢰기','실제','에어','시반','어감','위치','사운드','발열','상부','함께','독립','에는',
    '분할','세기','호감','용량','아날로그','로서','헤테로시클릴','할로겐','페닐','환자','메틸','클릴','결함','인부','당한','글로벌', '그라파이트','섬유',' 페이퍼','수지','컨텐츠','쿠션','컴포넌트','동부','하자','수명','회전체','트레이닝','분리','전기','결부','적의','조립','전력량','모바일',
    '유형', '카드','복조','정확도','위상','시계열','경계','토대','승선','레이어','분기','지게차','코일','정해진','커넥터','형상','개별','다비','입축','이스','전해','재활','냉각수','보시','픽셀','부상', '케이블','포트','의견',
    '인쇄회로기판','찾아내다', '변경','크기','신청', '출시', '스켈레톤', '역다', '슬롯', '로그', '프로토콜', '클라우드', '균열', '시험', '타겟', '콘크리트', '부품', '촬상', '피어', '접근', '통신망', '밀폐', '활성화', '화물', '건설', '비상', '상관', '적합하다', '나머지', '단일', '담당', '공개',
    '누설', '경비', '주행', '화질', '엔진', '사중', '광등', '구성요소', '태양광', '의미', '습도', '자체', '함수', '평균', '이하', '도출', '회로', '상이', '서로', '엮어', '차이나다', '점화', '케이스', '파형', '스파크', '노출', '성품',
    '차원', '데이터베이스', '전동차', '속성', '의복', '취득', '비컨', '긴급', '목표', '조업', '라인', '동키', '우레아', '효소', '계전기', '직접', '차단기', '작성', '가능성', '램프', '상체', '건전', '내시경', '굴절', '전자기기', '거치', '오프',
    '에너지', '프로세서', '콘텐츠', '로컬', '재부팅', '앵커', '링크', '암호', '브러쉬', '환기', '접지', '마모', '패킷', '화로', '동기화', '세션', '취출', '단위', '모범', '사례', '잠그다', '운항', '수준', '중인', '안테나', '종료', '레저', '요트'
    '변전소', '통풍', '파생', '판정', '배전', '구제', '탬덤', '덮개', '솔레노이드', '굳다', '밸브', '페이', '계류', '마리나', '가이드', '판별', '항해', '제어기', '불쾌', '그린', '분산', '이상인', '모두', '측량', '끼리', '과거', '마이크로', '실어', '변전', '검사', '용기', '광원', '화용', '내압 레이저', '앰프', '예상', '이더넷',
    '물건', '통과', '근무', '무인기', '의심', '컨트롤러', '스캔', '스크린도어', '통계', '송수', '소지', '공부', '도크', '배지', '기동', '상대', '청력', '마이크로폰', '장갑', '트래킹', '중앙', '인코더', '중계', '화기', '초당', '스펙', '에러', '중계기', '간의', '가변', '중량', '알고리즘',
    '해결', '예정', '카테고리', '시간대', '참조', '공유', '지도', '상승', '하락', '지속', '매핑', '성분', '체류', '장부', '자로', '조이다', '매우', '해상도', '나누다', '중요도', '낮다', '증가', '이면', '노랑', '외형', '스티커', '스트리밍', '인쇄', '콘덴서', '극상', '텍스트',
    '등록', '해지', '부하', '삽입', '신체', '조작', '임계', '진폭', '증폭', '레이블', '소형', '선의', '명칭', '제작', '경과', '탁월', '반영', '업로드', '플러스', '신후', '동시', '사항', '존재', '날짜', '종류', '혹은', '목록', '가볍다', '회전기', '이엠', '비아',
    '삼각', '육상', '원래', '마이너스', '보트', '측위', '제거', '문자', '노이즈', '베드', '및기', '라이더', '년도', '미래', '설계도', '따라서', '관심', '유무', '좌표', '주차', '이동속도', '사람인', '선정', '높다', '분수', '배수', '시동', '재질', '차속', '교신', '비정상', '배기가스', '회피', '확률', '이중',
    '수득', '대책', '측정값', '안전센', '압전센', '관계', '아니다', '부근', '축소', '행동', '차선', '후방', '보장', '소방차', '구급차', '정상', '장면', '중이', '컬러', '지정', '회의', '구별', '보안안', '우선적', '선적',
    '정보', '조명기구', '방법', '지표', '선박박', '의사', '선박', '양단', '장치치', '누수', '단말기', '데이터', '연료', '전지', '보안', '이상', '장치', '센서', '상태' , '변전소', '감지', '연동', '이탈', '리소스', '페이퍼', '레벨', '상점', '스트레스', '레이더', '재래', '시장', '표적', '중장비',
    '타이어', '프롬', '프트', '질환', '검색', '현실', '방호벽', '정비', '에서의', '추진', '자외선', '냉각', '화각', '주기', '넘다', '연구','바디','맞춤','일체', '후드', '밀착', '지표', '둘레', '보호복','커버', '매뉴얼', '교체', '협착', '시뮬레이션', '전기요금', '정기','인스턴스','슬라이딩','주위','공기','도우미','냉온','표면','화학물질','압구','웨이퍼','방열',
    '적정','자식','부들','보충','음원','각도','성공','이력','봉투','유효하다','웨이블릿','신세기','글래스','탄성','시침','피스톤','수리','벨트','매개변수','축의','비율','쓰다','플로우','생활','도르다','준비','모습','안착','역률','기구','잔류','측정기','가시광선','원수','홀더','에틸렌','프로필렌','헤드','하의','통한','레진',
    '최고점','저항','선단','차수','진출','하강','정화','불꽃','접착','정형','열전도','터널링','공작','반의','도포','베이스','음극','나타내다','엘리베이터','바이어스','기상','스팅','보정','움직임','익수','태깅','개체','수립','상계','동기','판독','안정','로드','목별','취약','지각','청각','증대','저하','소리','가산','도포','유압잭','귀걸이','풍력발전','표정',
    '수직','수평','헤드','데크','기자','각속도','평행','임베디드','로트','사이즈','스팅','펜싱','목별','상단','업데이트','대차','수습','페달','호강','일산화탄소','원점','이웃','예비','선단','반사경','수직','수평','요약','샘플링','감쇠','각의','재물','제약','스프링','기술자','보고','커패시터','신뢰','활선','차륜','블럭','직상','용단','입자','파싱','지하','목적',
    '재래','표적','시장','스트레스','상점','선박박','볼라드','교통','샘플','도로','밴드','변압기','포지셔닝','써비','검지','접점','대피소','발급','전도',
    
    '이미지','장력','펌웨어','영상','전계','순차','도어','중공','시트','트러스트','벤츄리','오존','넘버','자재','정체','청구','항','생','성하다','수질', '랙티브', '인터', '연관','결정','유지','시퀀스','자속','포맷', '설비', '안전', '제어', '분석',
    '''''
    '전극', '세트', '처리','패널','돌출', '이동', '감시', '알림', '재난','아웃렛','주민','수광',
    '단자','비아','웹페이지','기관', '편차', '취급', '음향', '한계', '적외선', '보수', '초기', '비행', '이후','원판','착용', '태그','스피커', '점수', '기록', '워드', '다음','오스','체온','축척','메시지','하우','소켓','위험관리','행태','세다','하중','위급','착탈','감싸다','날씨','산정','모용','마스크','상행위','이며','사진','진압','유저',
    '오류','제스쳐','물리','식품','벡터','응급','건축물'
    '''} 



In [420]:
# 표제어 추출기 설정

lemmatizer = WordNetLemmatizer()

In [421]:
# KoBERT 모델 로드
kobert_model = AutoModel.from_pretrained("skt/kobert-base-v1")

In [422]:
# 트리뱅크 형식의 품사 태그를 WordNet 형식에 맞게 변환

def wordnet_pos_tags_kor(treebank_tag):
    """Converts POS tags from Korean treebank format to WordNet format."""
    if treebank_tag.startswith('VA') or treebank_tag.startswith('VV'):
        return 'v'  # verb
    elif treebank_tag.startswith('N'):
        return 'n'  # noun
    elif treebank_tag.startswith('M'):
        return 'r'  # adverb (관형사, 부사)
    elif treebank_tag.startswith('XR') or treebank_tag.startswith('MM'):
        return 'a'  # adjective (어근, 관형사)
    else:
        return 'n'  # default to noun


In [423]:
okt = Okt()

In [424]:
# 특허 데이터 적용 (형태소 분석 & 불용어 제거)

import re

# 전처리
def preprocess_patent_summaries(summaries):

    preprocessed_summaries = []
    for summary in summaries:

        summary = re.sub(r'[^가-힣\s]', '', summary)  # 한글과 공백을 제외하고 모두 제거
        summary = re.sub(r'\s+', ' ', summary)  # 중복 공백 제거
        summary = summary.strip()

        # 형태소 분석 및 불용어 제거
        tokens = okt.morphs(summary, stem=True)  # 형태소 분석 및 어간 추출
        tokens = [token for token in tokens if token not in stopword]  # 불용어 제거

        # 전처리된 문서를 리스트에 추가
        preprocessed_summaries.append(" ".join(tokens))

    return preprocessed_summaries

In [425]:
# 표제어 추출
def extract_lemmatized_tokens(preprocessed_docs):
    lemmatized_docs = []
    for document in preprocessed_docs:
        tokens_with_pos = okt.pos(document, stem=True)  # 품사 태깅
        lemmatized_tokens = [token for token, tag in tokens_with_pos if wordnet_pos_tags_kor(tag)]
        lemmatized_docs.append(" ".join(lemmatized_tokens))

    return lemmatized_docs

In [590]:
# 특허 요약문 전처리
patent_prep = preprocess_patent_summaries(summ)
# print(patent_prep)

In [591]:
# 표제어 추출
lemmatized_patents = extract_lemmatized_tokens(patent_prep)
# print(lemmatized_patents)

In [592]:
# 동일한 파라미터 값에 대해 항상 동일한 값이 출력되도록 고정

import random

seed = 42

# Python 내장 랜덤 모듈 시드 설정
random.seed(seed)

# NumPy 시드 설정
np.random.seed(seed)

# PyTorch 시드 설정
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [622]:
# KoBERT 모델과 토크나이저 불러오기
#tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
#model = BertModel.from_pretrained('monologg/kobert')

# Vectorizer 모델 설정
vectorizer_model = CountVectorizer(ngram_range=(1, 1), stop_words=list(stopword)) 

# UMAP 모델 설정
umap_model = umap.UMAP(n_neighbors=800, n_components=10, min_dist=0, metric='cosine', random_state=seed) # , spread=3.0)

# HDBSCAN 모델 설정
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# cTF-IDF 모델 설정
ctfidf_model = ClassTfidfTransformer()

In [623]:
# BERTopic 모델 초기화 및 훈련
topic_model = BERTopic(
    language="korean",  # 언어 설정
    calculate_probabilities=True,  # 확률 계산 여부
    nr_topics='auto',  # 주제의 수 제한, auto로 해야 HDBSCAN 작동
    top_n_words=10,  # 각 주제의 상위 단어 수
    min_topic_size=70,  # 주제의 최소 크기
    vectorizer_model=vectorizer_model,  # 벡터화 모델
    embedding_model=kobert_model,  # 임베딩 모델
    umap_model=umap_model,  # UMAP 모델
    hdbscan_model=hdbscan_model,  # HDBSCAN 모델
    ctfidf_model=ctfidf_model,  # c-TFIDF 모델
    verbose=True  # 진행 상황 출력 여부
)

In [624]:
# 주제 모델 훈련
topics, probabilities = topic_model.fit_transform(lemmatized_patents)

2024-06-27 02:18:18,246 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 20/20 [00:16<00:00,  1.22it/s]
2024-06-27 02:18:37,124 - BERTopic - Embedding - Completed ✓
2024-06-27 02:18:37,140 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-27 02:18:39,659 - BERTopic - Dimensionality - Completed ✓
2024-06-27 02:18:39,659 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-27 02:18:39,678 - BERTopic - Cluster - Completed ✓
2024-06-27 02:18:39,678 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-27 02:18:39,713 - BERTopic - Representation - Completed ✓
2024-06-27 02:18:39,713 - BERTopic - Topic reduction - Reducing number of topics
2024-06-27 02:18:39,743 - BERTopic - Topic reduction - Reduced number of topics from 4 to 4


In [625]:
# 주제 모델 결과 출력
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,251,-1_카메라_모니터링_지능_무선,"[카메라, 모니터링, 지능, 무선, 관제, 예측, 경보, 인공, 안전모, 네트워크]",[지능 예방 경로 상 얼굴 얼굴 열 열 카메라 일정 움직 이다 실시간 추적 추적 카...
1,0,184,0_평가_안전모_경고_무선,"[평가, 안전모, 경고, 무선, 예측, 관제, 훈련, 실시간, 센싱, 생체]",[도 평가 노후 도 평가 평가 평가 평가 유사 평가 평가 평가 평가 평가 빈도 예측...
2,1,164,1_모니터링_무선_실시간_스위치,"[모니터링, 무선, 실시간, 스위치, 가상, 경보, 감지기, 카메라, 네트워크, 교육]",[비콘 증 지문 손 혈관 정맥 얼굴 생체 심박수 심박수 생체 생체인식 방문자 생체 ...
3,2,24,2_예측_로봇_레이저_카메라,"[예측, 로봇, 레이저, 카메라, 빅데이터, 관제, 원격, 인공, 실시간, 컨테이너]",[폭 예측 예측 예측 예측 예측 예측 예측 예측 예측 예측 예측 예측 설계 설계 예...


In [626]:
for topic_num in topic_model.get_topics():
    words = [word for word, _ in topic_model.get_topic(topic_num)] 
    print(f"Topic {topic_num}: {' '.join(words)}")

Topic -1: 카메라 모니터링 지능 무선 관제 예측 경보 인공 안전모 네트워크
Topic 0: 평가 안전모 경고 무선 예측 관제 훈련 실시간 센싱 생체
Topic 1: 모니터링 무선 실시간 스위치 가상 경보 감지기 카메라 네트워크 교육
Topic 2: 예측 로봇 레이저 카메라 빅데이터 관제 원격 인공 실시간 컨테이너


In [627]:
# 문서별 할당된 토픽
doc_topics = pd.DataFrame({"Document": patent_prep, "Topic": topics})
# print(doc_topics)

In [628]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from ugtm import eGTM, eGTC
import altair as alt

In [634]:
# GTM 모델 학습 및 변환 
# 주어진 항만 특허 데이터의 주제 Scope를 고려하여 적합한 차원 수 설정
gtm_model = eGTM(k=5, model="responsibilities").fit(probabilities)
gtm_means = eGTM(k=5, model="means").fit_transform(probabilities)
gtm_modes = eGTM(k=5, model="modes").fit_transform(probabilities)

#print(gtm_model)
print(gtm_means)
print(gtm_modes)

[[-9.85872950e-01  9.68066629e-01]
 [-4.84249919e-01  5.72449802e-01]
 [ 5.51820965e-01 -4.48163544e-01]
 ...
 [ 9.99524349e-01  4.99070804e-01]
 [-8.73244008e-01  1.19790810e-01]
 [ 4.99051612e-01  5.35925537e-04]]
[[-1.   1. ]
 [-0.5  0.5]
 [ 0.5 -0.5]
 ...
 [ 1.   0.5]
 [-1.   0.5]
 [ 0.5  0. ]]


In [691]:
# GTM 결과 시각화
dgtm_modes = pd.DataFrame(gtm_modes, columns=["x1", "x2"])
dgtm_modes["topic"] = doc_topics['Topic']

# GTM modes 차트 생성
gtm_modes_chart = alt.Chart(dgtm_modes).mark_circle().encode(
    x='x1',
    y='x2',
    color=alt.Color(
        'topic:N',
        scale=alt.Scale(domain=[-1, 0, 1, 2],
                        range=['purple', 'orchid', 'hotpink', 'pink']),
        legend=None
    ),
    size=alt.value(1500),
    tooltip=['x1', 'x2', 'topic:N']
).properties(
    title="GTM (modes)", width=400, height=400
)

# 데이터프레임 생성
dgtm_means = pd.DataFrame(gtm_means, columns=["x1", "x2"])
dgtm_means["topic"] = doc_topics['Topic']

# GTM means 차트 생성
gtm_means_chart = alt.Chart(dgtm_means).mark_circle().encode(
    x='x1',
    y='x2',
    color=alt.Color(
        'topic:N',
        scale=alt.Scale(domain=[-1, 0, 1, 2],
                        range=['purple', 'orchid', 'hotpink', 'pink']),
        legend=None
    ),
    size=alt.value(100),
    tooltip=['x1', 'x2', 'topic:N']
).properties(
    title="GTM (means)", width=400, height=400
)

In [692]:
# 두 시각화 결합
gtm_chart = gtm_means_chart | gtm_modes_chart
gtm_chart.display()

alt.HConcatChart(...)

In [637]:
# Construct GTM
gtm_all = eGTM(k=5, model="responsibilities").fit(probabilities)

gtm_modes_all = eGTM(k=5, model="modes").fit_transform(probabilities)

dgtm_modes_all = pd.DataFrame(gtm_modes_all, columns=["x1", "x2"])
# dgtm_modes["label"] = y

gtm_modes_all = alt.Chart(dgtm_modes_all).mark_circle().encode(
    x='x1',
    y='x2',
#    color=alt.Color('label:N',
#                    scale=alt.Scale(scheme='viridis')),
    size=alt.value(3000),
    tooltip=['x1','x2','label:N']
).properties(title = "GTM (modes)", width = 500, height = 500)

In [639]:
gtm_modes_all.display()

alt.Chart(...)

In [640]:
gtm_df = pd.DataFrame(probabilities)

# transform을 적용할 함수 정의
def custom_transform(x):
    return x + 1

# 각 요소에 함수를 적용하여 transform 수행
gtm_inverse = gtm_df.apply(custom_transform)

# print(gtm_inverse)

transformed = gtm_all.transform(gtm_df)
gtm = gtm_all.optimizedModel.matY

gtm_res = pd.DataFrame(gtm)

print(gtm_res)

         0         1         2         3         4         5         6   \
0  0.997153  1.005743  0.804985  0.652834  0.535686  0.690672  0.733221   
1 -0.000229  0.004013  0.075923  0.139697  0.118090  0.119503  0.120173   
2  0.000183  0.005316  0.103237  0.174790  0.191787  0.154850  0.112341   

         7         8         9   ...        15        16        17        18  \
0  0.565144  0.451883  0.442677  ...  0.114114  0.154075  0.249744  0.231352   
1  0.179978  0.223736  0.157164  ...  0.961873  0.606685  0.415389  0.307073   
2  0.132284  0.156704  0.173400  ... -0.667320  0.107748  0.151384  0.185581   

         19        20        21        22        23        24  
0  0.001390  0.001783  0.096122  0.129528  0.185986  0.286248  
1 -0.000667  0.996901  0.802521  0.682797  0.521777  0.083259  
2  0.990047 -0.002392  0.086828  0.079935  0.139811  0.412966  

[3 rows x 25 columns]


In [641]:
gtm_res.transpose().to_csv("C:/Users/user/Desktop/KoBERT/gtm_inverse_v.0.1.4.csv")